In [1]:
import torch
import tensorflow as tf
from transformers import RobertaTokenizer, RobertaModel, TFRobertaModel

2023-05-09 13:09:14.260342: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-09 13:09:15.159928: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
MODEL_NAME = "roberta-base"

In [18]:
tokenizer = RobertaTokenizer.from_pretrained(MODEL_NAME)
embedding_model = RobertaModel.from_pretrained(MODEL_NAME)
tf_embedding_model = TFRobertaModel.from_pretrained(MODEL_NAME)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClass

In [29]:
MAX_LEN = 64

def embed(input: str, tensor_type="torch"):

    tokens = tokenizer.encode(input, add_special_tokens=True, max_length=MAX_LEN,
                                return_token_type_ids=True, padding="max_length",
                                truncation=True)
  
    if tensor_type == "torch":
        input_ids = torch.tensor(tokens).unsqueeze(dim=0)
        outputs = embedding_model(input_ids)
        embeddings = outputs.last_hidden_state.mean(dim=1)
        
    else:
        input_ids = tf.constant(tokens, shape=(1, 64))
        outputs = tf_embedding_model(input_ids)
        embeddings = tf.reduce_mean(outputs.last_hidden_state, axis=1)
    
    return embeddings

In [30]:
e = embed("You say what's up we say wagwan", tensor_type="tf")
e.shape

TensorShape([1, 768])